In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import requests
import nltk
from textblob import TextBlob as tb
from langdetect import detect 

In [150]:
data=pd.read_excel("Data Analyst - Test Data.xlsx")

In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Review    6393 non-null   object        
 1   date      6448 non-null   datetime64[ns]
 2   Location  1711 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 100.8+ KB


In [152]:
data['Location'].isna().sum()

4737

In [153]:
data.head(5)

,Review,date,Location
0,I was very impressed with the resort.\n Great ...,2019-08-20,Sebastian
1,The rooms were nice the outside needs work als...,2019-08-20,Los Angeles
2,Great location! I have stayed at this hotel on...,2019-08-20,Georgia
3,The hotel was adequate for my stay. The strips...,2019-08-20,NaN
4,"Great location, room was large and spacious. P...",2019-08-19,Palm Harbor


In [155]:
data['Review'].fillna(" ",inplace=True)

In [156]:
data['Review']=data['Review'].apply(lambda x:x.lower())

In [158]:
def lang(x):
    try:
        return detect(x)
    except:
        return "UnKnown"

In [160]:
data['Language']=data['Review'].apply(lambda x: lang(x))

In [162]:
data['Review']=data['Review'].apply(lambda x:re.findall("\w+",x))

In [163]:
resp1=requests.get("https://ptrckprry.com/course/ssd/data/positive-words.txt")

In [164]:
raw_data1=resp1.content[1535:]

In [165]:
positive_words=raw_data1.decode().split('\n')[3:]

In [166]:
resp2=requests.get("https://ptrckprry.com/course/ssd/data/negative-words.txt")

In [167]:
raw_data2=resp2.content[1550:]

In [168]:
negative_words=raw_data2.decode(encoding='latin-1').split("\n")[1:]

In [169]:
from nltk.corpus import stopwords

In [170]:
stopwords=stopwords.words("english")

In [171]:
def remove(x):
    for i in x:
        if i.strip() in stopwords:
            x.remove(i)
    return x        

In [172]:
data['Review']=data['Review'].apply(lambda x: remove(x))

In [173]:
data.head(5)

,Review,date,Location,Language
0,"[was, impressed, the, resort, great, staff, th...",2019-08-20,Sebastian,en
1,"[rooms, nice, outside, needs, work, also, free...",2019-08-20,Los Angeles,en
2,"[great, location, have, stayed, this, hotel, m...",2019-08-20,Georgia,en
3,"[hotel, adequate, my, stay, strips, bathtub, u...",2019-08-20,NaN,en
4,"[great, location, room, large, spacious, parki...",2019-08-19,Palm Harbor,en


In [174]:
positive_score=[]
negative_score=[]
polarity_score=[]
for i in data['Review']:
    pc=0
    nc=0
    for j in i:
        if j in positive_words:
            pc+=1
        elif j in negative_words:
            nc+=1
    positive_score.append(pc)  
    negative_score.append(nc)
    polarity_score.append((pc-nc))

In [175]:
data['positive_score']=positive_score
data['negative_score']=negative_score
data['polarity_score']=polarity_score

In [176]:
data[data['polarity_score']==data['polarity_score'].min()]

,Review,date,Location,Language,positive_score,negative_score,polarity_score
949,"[ursprünglich, gutes, angebot, später, ernücht...",2019-05-17,NaN,de,1,15,-14
3766,"[arrived, location, checked, my, room, receive...",2018-11-26,"New York, NY",en,0,14,-14


In [177]:
data.to_csv('final_data.csv')

In [13]:
data=pd.read_csv('final_data.csv',index_col=0)

In [14]:
data['Language'].nunique()

23

### inference 1 :- we have 22 diffrent types of language reviews.

In [15]:
data['Language'].value_counts()

en         5861
es          169
de           83
fr           75
UnKnown      56
it           37
ja           37
pt           32
nl           25
da           19
no           10
ko            9
sv            8
et            5
fi            4
zh-cn         4
af            4
ca            3
pl            2
ru            2
ar            1
zh-tw         1
he            1
Name: Language, dtype: int64

### inference 2 :- 90.9 % reviews is in english language

In [16]:
((data['Language']=='en').sum()/data.__len__())*100

90.89640198511167

In [17]:
m=data['polarity_score'].max()

In [18]:
data[data['polarity_score']==m]

,Review,date,Location,Language,positive_score,negative_score,polarity_score
2660,"['husband', 'pleased', 'our', 'stay', 'new', '...",2019-01-28,"Chicago,Il",en,26,4,22


### Most positive review is from chicago on 28th january 2019

In [19]:
data['date']=data['date'].astype(str)

In [20]:
data['date'].apply(lambda x:x.split('-')[0]).value_counts()

2019    3856
2018    2592
Name: date, dtype: int64

In [21]:
n=data['polarity_score'].min()

In [22]:
data[data['polarity_score']==n]

,Review,date,Location,Language,positive_score,negative_score,polarity_score
949,"['ursprünglich', 'gutes', 'angebot', 'später',...",2019-05-17,NaN,de,1,15,-14
3766,"['arrived', 'location', 'checked', 'my', 'room...",2018-11-26,"New York, NY",en,0,14,-14


#### There are two most negative reviews one is in english language from NewYORK


In [28]:
en_data=data[data['Language']=='en']

In [34]:
en_data['Month']=en_data['date'].apply(lambda x:x.split('-')[1].strip())

<ipython-input-34-a801e7c88532>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_data['Month']=en_data['date'].apply(lambda x:x.split('-')[1].strip())


In [39]:
en_data['year']=en_data['date'].apply(lambda x:x.split('-')[0].strip())

<ipython-input-39-12250379c8ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_data['year']=en_data['date'].apply(lambda x:x.split('-')[0].strip())


In [51]:
en_data.groupby(['year','Month'])['Review'].count()

year  Month
2018  08       396
      09       430
      10       469
      11       461
      12       566
2019  01       454
      02       390
      03       557
      04       537
      05       471
      06       338
      07       524
      08       268
Name: Review, dtype: int64

#### Highest  No.  of reviews is in the december month. 

In [53]:
en_data

,Review,date,Location,Language,positive_score,negative_score,polarity_score,Month,year
0,"['was', 'impressed', 'the', 'resort', 'great',...",2019-08-20,Sebastian,en,5,0,5,08,2019
1,"['rooms', 'nice', 'outside', 'needs', 'work', ...",2019-08-20,Los Angeles,en,4,0,4,08,2019
2,"['great', 'location', 'have', 'stayed', 'this'...",2019-08-20,Georgia,en,1,0,1,08,2019
3,"['hotel', 'adequate', 'my', 'stay', 'strips', ...",2019-08-20,NaN,en,3,4,-1,08,2019
4,"['great', 'location', 'room', 'large', 'spacio...",2019-08-19,Palm Harbor,en,5,0,5,08,2019
...,...,...,...,...,...,...,...,...,...
6443,"['staff', 'friendly', 'frank', 'chef', 'buffet...",2019-07-12,United States of America,en,7,1,6,07,2019
6444,"['internet', 'barely', 'worked', 'didn', 'work']",2019-07-12,United States of America,en,2,0,2,07,2019
6445,"['location', 'great', 'bed', 'the', 'uncomfort...",2019-07-10,United States of America,en,1,1,0,07,2019
6446,"['were', 'early', 'check', 'and', 'staff', 've...",2019-07-10,United States of America,en,0,0,0,07,2019


In [57]:
en_data[en_data['Location']=='United States of America']

,Review,date,Location,Language,positive_score,negative_score,polarity_score,Month,year
6267,"['was', 'close', 'what', 'were', 'for', 'nothi...",2019-08-19,United States of America,en,0,0,0,08,2019
6271,"['location', 'excellent', 'td', 'garden', 'wel...",2019-08-18,United States of America,en,4,0,4,08,2019
6272,"['nothing', 'complain', 'location', 's', 'clos...",2019-08-18,United States of America,en,4,4,0,08,2019
6273,"['location', 'pricing', 'parking', 'extra', 'f...",2019-08-18,United States of America,en,0,0,0,08,2019
6274,"['parking', 'the', 'price', 'pay', 'no', 'park...",2019-08-18,United States of America,en,0,0,0,08,2019
...,...,...,...,...,...,...,...,...,...
6443,"['staff', 'friendly', 'frank', 'chef', 'buffet...",2019-07-12,United States of America,en,7,1,6,07,2019
6444,"['internet', 'barely', 'worked', 'didn', 'work']",2019-07-12,United States of America,en,2,0,2,07,2019
6445,"['location', 'great', 'bed', 'the', 'uncomfort...",2019-07-10,United States of America,en,1,1,0,07,2019
6446,"['were', 'early', 'check', 'and', 'staff', 've...",2019-07-10,United States of America,en,0,0,0,07,2019
